# this is the final test file  

In [ ]:
import os
import random
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
from torchvision.models import resnet18
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# just calculates the train and test acc
# Set paths
model_path = "path to model"
train_path = "path to train folder"
test_path = "path to test folder"

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18()

# Try loading the state dict or the entire model
checkpoint = torch.load(model_path)
if isinstance(checkpoint, dict):
    model.load_state_dict(checkpoint)
else:
    model = checkpoint

model.to(device)
model.eval()

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match model input size, adjust if necessary
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = ImageFolder(root=train_path, transform=transform)
test_dataset = ImageFolder(root=test_path, transform=transform)

# Get class indices
classes = train_dataset.classes
selected_classes = random.sample(classes, 10)
class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
selected_indices = [class_to_idx[cls] for cls in selected_classes]

# Filter datasets for selected classes
def get_class_indices(dataset, class_indices):
    return [idx for idx, (_, label) in enumerate(dataset) if label in class_indices]

train_indices = get_class_indices(train_dataset, selected_indices)
test_indices = get_class_indices(test_dataset, selected_indices)

train_subset = Subset(train_dataset, train_indices)
test_subset = Subset(test_dataset, test_indices)

# Create DataLoaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)

# Evaluation function
def evaluate(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Evaluate on train and test subsets
train_accuracy = evaluate(train_loader, model, device)
test_accuracy = evaluate(test_loader, model, device)

print(f"Train Accuracy on selected classes: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy on selected classes: {test_accuracy * 100:.2f}%")

In [ ]:
# this code shows the predicted and actual photos 
# Set paths
model_path = "path to model"
train_path = "path to train folder"
test_path = "path to test folder"

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18()

# Try loading the state dict or the entire model
checkpoint = torch.load(model_path)
if isinstance(checkpoint, dict):
    model.load_state_dict(checkpoint)
else:
    model = checkpoint

model.to(device)
model.eval()

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match model input size, adjust if necessary
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = ImageFolder(root=train_path, transform=transform)
test_dataset = ImageFolder(root=test_path, transform=transform)

# Get class indices
classes = train_dataset.classes
selected_classes = random.sample(classes, 10)
class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
selected_indices = [class_to_idx[cls] for cls in selected_classes]

# Filter datasets for selected classes
def get_class_indices(dataset, class_indices):
    return [idx for idx, (_, label) in enumerate(dataset) if label in class_indices]

train_indices = get_class_indices(train_dataset, selected_indices)
test_indices = get_class_indices(test_dataset, selected_indices)

train_subset = Subset(train_dataset, train_indices)
test_subset = Subset(test_dataset, test_indices)

# Create DataLoaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=False)  # No shuffle to keep image order
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)    # No shuffle to keep image order

# Evaluation function with detailed prediction printing
def evaluate_and_print(loader, model, device, classes, dataset_name):
    model.eval()
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            for i in range(len(images)):
                image_idx = batch_idx * loader.batch_size + i
                print(f"{dataset_name} - Class {classes[labels[i]]}: Image {image_idx} predicted as {classes[predicted[i]]}")

# Evaluate on train and test subsets
print("Train Predictions:")
evaluate_and_print(train_loader, model, device, train_dataset.classes, "Train")

print("\nTest Predictions:")
evaluate_and_print(test_loader, model, device, test_dataset.classes, "Test")

In [ ]:
# this one shows the wrong prediction photos
# Set paths
model_path = "path to model"
train_path = "path to train folder"
test_path = "path to test folder"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18()

# Try loading the state dict or the entire model
checkpoint = torch.load(model_path)
if isinstance(checkpoint, dict):
    model.load_state_dict(checkpoint)
else:
    model = checkpoint

model.to(device)
model.eval()

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match model input size, adjust if necessary
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = ImageFolder(root=train_path, transform=transform)
test_dataset = ImageFolder(root=test_path, transform=transform)

# Get class indices
classes = train_dataset.classes
selected_classes = random.sample(classes, 4)
class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
selected_indices = [class_to_idx[cls] for cls in selected_classes]

# Filter datasets for selected classes
def get_class_indices(dataset, class_indices):
    return [idx for idx, (_, label) in enumerate(dataset) if label in class_indices]

train_indices = get_class_indices(train_dataset, selected_indices)
test_indices = get_class_indices(test_dataset, selected_indices)

train_subset = Subset(train_dataset, train_indices)
test_subset = Subset(test_dataset, test_indices)

# Create DataLoaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=False)  # No shuffle to keep image order
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)    # No shuffle to keep image order

# Function to display images
def show_image(image, title):
    img = image.permute(1, 2, 0).cpu().numpy()  # Convert from Tensor to NumPy array and adjust dimensions
    img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])  # Unnormalize
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.title(title)
    plt.axis('off')
    plt.show()

# Evaluation function with detailed prediction printing
def evaluate_and_print(loader, model, device, classes, dataset_name, dataset):
    model.eval()
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            for i in range(len(images)):
                image_idx = batch_idx * loader.batch_size + i
                actual_class = classes[labels[i]]
                predicted_class = classes[predicted[i]]
                image_path, _ = dataset.samples[train_indices[image_idx] if dataset_name == "Train" else test_indices[image_idx]]
                image_name = os.path.basename(image_path)
                print(f"{dataset_name} - Class {actual_class}: Image '{image_name}' predicted as {predicted_class}")
                
                if actual_class != predicted_class:
                    actual_image = images[i].cpu()
                    show_image(actual_image, f"Actual: {actual_class}, Predicted: {predicted_class}")

# Evaluate on train and test subsets
print("Train Predictions:")
evaluate_and_print(train_loader, model, device, train_dataset.classes, "Train", train_dataset)

print("\nTest Predictions:")
evaluate_and_print(test_loader, model, device, test_dataset.classes, "Test", test_dataset)